# Regresion logistica: Deteccion de SPAM

En este ejecicio se muestran los fundamentos de la regresion logistica planteando uno de los primeros problemas que fueron solucionadeos mediante el uso de tecnicas de Machine Learning: la Deteccion de SPAM

## Enunciado del ejercicio
Se propone la construccion de un sistema de aprendizaje automatico capas de predecir si un correo determinado se corresponde con un correo SPAM o no, para ello se utilizara el siguente DataSet:

##### [2007 TREC Public Spam Corpus](https://plg.uwaterloo.ca/~gvcormac/treccorpus07/)


The corpus trec07p contains 75,419 messages:

    25220 ham
    50199 spam

These messages constitute all the messages delivered to a particular
server between these dates:

    Sun, 8 Apr 2007 13:07:21 -0400
    Fri, 6 Jul 2007 07:04:53 -0400

In [1]:
# En esta clase se facilita el procesamiento de correos electronicos que poseen codigo HTML.

from html.parser import HTMLParser
class MLStripper(HTMLParser):
    def __init__(self):
        self.reset()
        self.strict = False
        self.convert_charrefs = True
        self.fed = []
        
    def handle_data(self, d):
        self.fed.append(d)
        
    def get_data(self):
        return ''.join(self.fed)

In [2]:
# Esta funcion se encarga de eliminar los tags HTML que se encuentren en el texto de los correos electronicos

def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

In [3]:
#Ejemplo de eliminacion de los tags HTML de un texto
t = '<tr><td aling="left"><ahref="../../issues/51/16.html#article">Phrack World News </a><td>'
strip_tags(t)

'Phrack World News '

Ademas de eliminar los posibles tags HTML que se encuentran en el correo electronico deben reslizarse ptras opciones para evitar que los mensajes contegan ruido inecesario. Entre ellas se encuentra la eliminacion de los signos de puntuacion, eliminacion de los posibles campos de corroeo electrinico que no sean relevantes o eliminacion de los afijos de un a palabra manteniendo unicamente la raiz de la misma (Stemming). La clase que se muestra a continuacion realiza estas transformaciones.

In [4]:
import email
import string
import nltk




class Parser:
    def __init__(self):
        self.stemmer = nltk.PorterStemmer()
        self.stopwords = set(nltk.corpus.stopwords.words('english'))
        self.punctuation = list(string.punctuation)
        
    def parse(self, email_path):
        """Parse an email."""
        with open(email_path, errors='ignore') as e:
            msg = email.message_from_file(e)
        return None if not msg else self.get_email_content(msg)
    
    def get_email_content(self, msg):
        """Extract the email content."""
        subject = self.tokenize(msg['Subject'])if msg['Subject']else[]
        body = self.get_email_body(msg.get_payload(),
                                   msg.get_content_type())
        content_type = msg.get_content_type()
        # return the content of the email
        return {"subject": subject, 
                "body": body,
                "content_type": content_type}
    
    def get_email_body(self, payload, content_type):
        """Extract the body of the email."""
        body = []
        if type(payload) is str and content_type == 'text/plain':
            return self.tokenize(payload)
        elif type(payload) is str and content_type == 'text/html':
            return self.tokenize(strip_tags(payload))
        elif type(payload) is list:
            for p in payload:
                body += self.get_email_body(p.get_payload(),
                                            p.get_content_type())
        return body
            
    def tokenize(self, text):
        """Transform a text string in tokens. Perform two main actions, clean the punctuation symbols and do stemming of the text"""
        for c in self.punctuation:
            text = text.replace(c," ")
            
        text = text.replace("\t", " ")
        text = text.replace("\n", " ")
        tokens = list(filter(None, text.split(" ")))
        #stemming of tokens
        return [self.stemmer.stem(w) for w in tokens if w not in self.stopwords]

In [5]:
inmail = open("datasets/datasets/trec07p/data/inmail.1").read()
print(inmail)

From RickyAmes@aol.com  Sun Apr  8 13:07:32 2007
Return-Path: <RickyAmes@aol.com>
Received: from 129.97.78.23 ([211.202.101.74])
	by speedy.uwaterloo.ca (8.12.8/8.12.5) with SMTP id l38H7G0I003017;
	Sun, 8 Apr 2007 13:07:21 -0400
Received: from 0.144.152.6 by 211.202.101.74; Sun, 08 Apr 2007 19:04:48 +0100
Message-ID: <WYADCKPDFWWTWTXNFVUE@yahoo.com>
From: "Tomas Jacobs" <RickyAmes@aol.com>
Reply-To: "Tomas Jacobs" <RickyAmes@aol.com>
To: the00@speedy.uwaterloo.ca
Subject: Generic Cialis, branded quality@ 
Date: Sun, 08 Apr 2007 21:00:48 +0300
X-Mailer: Microsoft Outlook Express 6.00.2600.0000
MIME-Version: 1.0
Content-Type: multipart/alternative;
	boundary="--8896484051606557286"
X-Priority: 3
X-MSMail-Priority: Normal
Status: RO
Content-Length: 988
Lines: 24

----8896484051606557286
Content-Type: text/html;
Content-Transfer-Encoding: 7Bit

<html>
<body bgcolor="#ffffff">
<div style="border-color: #00FFFF; border-right-width: 0px; border-bottom-width: 0px; margin-bottom: 0px;" align="

##### Parsin del correo electronico

In [6]:
p = Parser()
p.parse("datasets/datasets/trec07p/data/inmail.1")

{'subject': ['gener', 'ciali', 'brand', 'qualiti'],
 'body': ['do',
  'feel',
  'pressur',
  'perform',
  'rise',
  'occas',
  'tri',
  'viagra',
  'anxieti',
  'thing',
  'past',
  'back',
  'old',
  'self'],
 'content_type': 'multipart/alternative'}

#####  Lectura  del indice 
Estas funciones complementarias se encargan de cargar en memoria la ruta de cada correo electronico y su etiqueta correspondiente. {spam y ham}

In [7]:
index = open("datasets/datasets/trec07p/full/index").readlines()
index

['spam ../data/inmail.1\n',
 'ham ../data/inmail.2\n',
 'spam ../data/inmail.3\n',
 'spam ../data/inmail.4\n',
 'spam ../data/inmail.5\n',
 'spam ../data/inmail.6\n',
 'spam ../data/inmail.7\n',
 'spam ../data/inmail.8\n',
 'spam ../data/inmail.9\n',
 'ham ../data/inmail.10\n',
 'spam ../data/inmail.11\n',
 'spam ../data/inmail.12\n',
 'spam ../data/inmail.13\n',
 'spam ../data/inmail.14\n',
 'spam ../data/inmail.15\n',
 'spam ../data/inmail.16\n',
 'spam ../data/inmail.17\n',
 'spam ../data/inmail.18\n',
 'spam ../data/inmail.19\n',
 'ham ../data/inmail.20\n',
 'ham ../data/inmail.21\n',
 'spam ../data/inmail.22\n',
 'spam ../data/inmail.23\n',
 'spam ../data/inmail.24\n',
 'spam ../data/inmail.25\n',
 'spam ../data/inmail.26\n',
 'spam ../data/inmail.27\n',
 'spam ../data/inmail.28\n',
 'ham ../data/inmail.29\n',
 'spam ../data/inmail.30\n',
 'ham ../data/inmail.31\n',
 'spam ../data/inmail.32\n',
 'spam ../data/inmail.33\n',
 'ham ../data/inmail.34\n',
 'spam ../data/inmail.35\n',
 

In [8]:
import os

DATASET_PATH = "datasets/datasets/trec07p"

def parse_index(path_to_index, n_elements):
    ret_indexes = []
    index = open(path_to_index).readlines()
    for i in range(n_elements):
        mail = index[i].split(" ../")
        label = mail[0]
        path = mail[1][:-1]
        ret_indexes.append({"label": label,
                           "email_path": os.path.join(DATASET_PATH, path)})
        
    return ret_indexes

In [9]:
def parse_email(index):
    p = Parser()
    pmail = p.parse(index["email_path"])
    return pmail, index["label"]

In [10]:
indexes = parse_index("datasets/datasets/trec07p/full/index", 10)
indexes

[{'label': 'spam', 'email_path': 'datasets/datasets/trec07p/data/inmail.1'},
 {'label': 'ham', 'email_path': 'datasets/datasets/trec07p/data/inmail.2'},
 {'label': 'spam', 'email_path': 'datasets/datasets/trec07p/data/inmail.3'},
 {'label': 'spam', 'email_path': 'datasets/datasets/trec07p/data/inmail.4'},
 {'label': 'spam', 'email_path': 'datasets/datasets/trec07p/data/inmail.5'},
 {'label': 'spam', 'email_path': 'datasets/datasets/trec07p/data/inmail.6'},
 {'label': 'spam', 'email_path': 'datasets/datasets/trec07p/data/inmail.7'},
 {'label': 'spam', 'email_path': 'datasets/datasets/trec07p/data/inmail.8'},
 {'label': 'spam', 'email_path': 'datasets/datasets/trec07p/data/inmail.9'},
 {'label': 'ham', 'email_path': 'datasets/datasets/trec07p/data/inmail.10'}]

# Preprocesamiento del DataSet

con las funciones presentadas anetrioremente se permite la lectura de los correos electronicos de maenra programatica y el procesamieto de los mismos para eliminar aquellos componentes que no resultan de ultilidad para la deteccin de correos de SPAM. Sin emabrgo cada uno de los correos sigue estando representado por on diccionario de Python con una serie de palabras.

In [11]:
# Cargar el indice y las etiquetas en memoria
index = parse_index("datasets/datasets/trec07p/full/index", 1)

In [12]:
# leemos el primer correo
import os 
open(index[0]["email_path"]).read()

'From RickyAmes@aol.com  Sun Apr  8 13:07:32 2007\nReturn-Path: <RickyAmes@aol.com>\nReceived: from 129.97.78.23 ([211.202.101.74])\n\tby speedy.uwaterloo.ca (8.12.8/8.12.5) with SMTP id l38H7G0I003017;\n\tSun, 8 Apr 2007 13:07:21 -0400\nReceived: from 0.144.152.6 by 211.202.101.74; Sun, 08 Apr 2007 19:04:48 +0100\nMessage-ID: <WYADCKPDFWWTWTXNFVUE@yahoo.com>\nFrom: "Tomas Jacobs" <RickyAmes@aol.com>\nReply-To: "Tomas Jacobs" <RickyAmes@aol.com>\nTo: the00@speedy.uwaterloo.ca\nSubject: Generic Cialis, branded quality@ \nDate: Sun, 08 Apr 2007 21:00:48 +0300\nX-Mailer: Microsoft Outlook Express 6.00.2600.0000\nMIME-Version: 1.0\nContent-Type: multipart/alternative;\n\tboundary="--8896484051606557286"\nX-Priority: 3\nX-MSMail-Priority: Normal\nStatus: RO\nContent-Length: 988\nLines: 24\n\n----8896484051606557286\nContent-Type: text/html;\nContent-Transfer-Encoding: 7Bit\n\n<html>\n<body bgcolor="#ffffff">\n<div style="border-color: #00FFFF; border-right-width: 0px; border-bottom-width: 0

In [13]:
# parsear el primer correo
mail, label = parse_email(index[0])
print("El ecorreo es: ", label, "\n")
print(mail)

El ecorreo es:  spam 

{'subject': ['gener', 'ciali', 'brand', 'qualiti'], 'body': ['do', 'feel', 'pressur', 'perform', 'rise', 'occas', 'tri', 'viagra', 'anxieti', 'thing', 'past', 'back', 'old', 'self'], 'content_type': 'multipart/alternative'}


El algoritmo de Regresin Logistica no es capaz de ingeriri texto vomo parte del DataSet. Por lo tanto, deben aplicarse una serie de funciones adisionales que trasformen el texto de los correos electronicos parceados en una representacion numerica.

### Aplicacion de CountVectorizer

In [14]:
from sklearn.feature_extraction.text import CountVectorizer
#Preparacion del email en una cadena de texto.
prep_email = [" ".join(mail['subject']) + " ".join(mail['body'])]

vectorizer = CountVectorizer()
X = vectorizer.fit(prep_email)
print("e-mail: ", prep_email, "\n")
print("Caracteristicas de entrada: ", vectorizer.get_feature_names_out())

e-mail:  ['gener ciali brand qualitido feel pressur perform rise occas tri viagra anxieti thing past back old self'] 

Caracteristicas de entrada:  ['anxieti' 'back' 'brand' 'ciali' 'feel' 'gener' 'occas' 'old' 'past'
 'perform' 'pressur' 'qualitido' 'rise' 'self' 'thing' 'tri' 'viagra']


In [15]:
X = vectorizer.transform(prep_email)
print("\nValues: \n", X.toarray())


Values: 
 [[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]]


#### Aplicacion de OneHotEncoding

In [16]:
from sklearn.preprocessing import OneHotEncoder

prep_email = [[w] for w in mail['subject'] + mail['body']]
enc = OneHotEncoder(handle_unknown = 'ignore')
X = enc.fit_transform(prep_email)
print("Features:\n ", enc.get_feature_names_out(), "\n")
print("\nValues:\n", X.toarray())

Features:
  ['x0_anxieti' 'x0_back' 'x0_brand' 'x0_ciali' 'x0_do' 'x0_feel' 'x0_gener'
 'x0_occas' 'x0_old' 'x0_past' 'x0_perform' 'x0_pressur' 'x0_qualiti'
 'x0_rise' 'x0_self' 'x0_thing' 'x0_tri' 'x0_viagra'] 


Values:
 [[0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0

#### Funciones auxiliares preprosesamiento del DataSet

In [17]:
def create_prep_dataset(index_path, n_elements):
    X =[]
    y = []
    indexes = parse_index(index_path, n_elements)
    for i in range(n_elements):
        print("\rParsing email: {0}".format(i+1), end = '')
        mail, label = parse_email(indexes[i])
        X.append(" ".join(['subject']) + " ".join(mail['body']))
        y.append(label)
    return X, y    
    

In [26]:
# Leer unicamente un subconjunto de 1000 correso electronicos
X_train, y_train = create_prep_dataset("datasets/datasets/trec07p/full/index", 1000)
X_train

Parsing email: 1000

['subjectdo feel pressur perform rise occas tri viagra anxieti thing past back old self',
 'subjecthi updat gulu i check mirror it seem littl typo debian readm file exampl http gulu usherbrook ca debian readm ftp ftp fr debian org debian readm test lenni access releas dist test the current test develop snapshot name etch packag test unstabl pass autom test propog releas etch replac lenni like readm html yan morin consult en logiciel libr yan morin savoirfairelinux com 514 994 1556 to unsubscrib email debian mirror request list debian org subject unsubscrib troubl contact listmast list debian org',
 'subjectmega authenticv i a g r a discount pricec i a l i s discount pricedo miss it click http www moujsjkhchum com authent viagra mega authenticv i a g r a discount pricec i a l i s discount pricedo miss it click',
 'subjecthey billi realli fun go night talk said felt insecur manhood i notic toilet quit small area worri websit i tell secret weapon extra 3 inch trust girl love bigger one i 

Aprendizaje Donald Hebb
Matematicamente se puede dar el aprendizaje de las celulas
A.S
Datos etiquetados
Aprendizaje Basado en Modelos
Realizar predicciones
DATASET-> Algoritmos de aprendizaje (f(x)) -> h entra X y sale Y(prediccion)  MODELO DE ML

x = variable de  entrada
y = variable de salida
(x, y) ejemplo de entrenamiento
si conozco a p por lo tanto conozco a q (ley de agregacion)
(Featuras)             | (target(values))             |
Num. Equipos afectados | $Costo            |          |
i1 1000                | $10000            | DATASET  | DATA FRAME
i2 1500                | $20000            |          |
i3  500                | $55000            |          |
Im  200                | $2500             |

h0(x) = teta0 + teta1x    (parametros del modelo)
y = mx + n
h0(x) = teta 1 + teta0
Se evaluan los datos que legan con el mismo modelo matematico
X(univariable)
(x1, x2, x3...xn) = multivariable
h0(x) = teta0 + teta1 + teta1x1 +...... tetan xn

##### Aplicar vectorizacion a los datos

In [27]:
vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(X_train)

In [28]:
print(X_train.toarray())
print("\n Feactures", len(vectorizer.get_feature_names_out()))

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [2 0 0 ... 0 0 0]]

 Feactures 17296


In [29]:
import pandas as pd
pd.DataFrame(X_train.toarray(), columns = [vectorizer.get_feature_names_out()])

,00,000,0000,000000,000000000,000099,0000ff,0001171749,0001pt,000m,...,绰020,绰۹ϵͳctsƽ,肾ǝvă,鏗ėvłq,饻jwk,鵵χ,낢ȏglgwă,뼰ʱϵ,쫷ƹư,쳵㽻ѣ
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [30]:
y_train

['spam',
 'ham',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'ham',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'ham',
 'ham',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'ham',
 'spam',
 'ham',
 'spam',
 'spam',
 'ham',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'ham',
 'ham',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'ham',
 'spam',
 'ham',
 'spam',
 'spam',
 'ham',
 'spam',
 'spam',
 'spam',
 'ham',
 'ham',
 'spam',
 'spam',
 'spam',
 'ham',
 'ham',
 'ham',
 'spam',
 'ham',
 'spam',
 'ham',
 'spam',
 'spam',
 'spam',
 'ham',
 'spam',
 'spam',
 'ham',
 'spam',
 'ham',
 'spam',
 'spam',
 'ham',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'ham',
 'spam',
 'spam',
 'spam'

#### Entrenamiento del Algoritmo de regresion logistica con el DataSet preprocesado.

In [32]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression()
clf.fit(X_train, y_train)

LogisticRegression()

#  4. Prediccion

In [37]:
# Lectura de un DataSet de correos nuevos .
# Leer 150 de nuestro DataSet y quedarnos unicamente con los 500 ultimos correos electronicos, los cuales no se han utilizado
# para entrenar el algoritmo
X, y = create_prep_dataset("datasets/datasets/trec07p/full/index", 150)
X_test = X[100:]
y_test = y[100:]

Parsing email: 150

####  Preprocesamiento de los correos electronicos cin el vectorizado creado anetriormente

In [38]:
X_test = vectorizer.transform(X_test)

In [39]:
y_pred = clf.predict(X_test)
y_pred

array(['spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam',
       'spam', 'spam', 'ham', 'spam', 'spam', 'spam', 'spam', 'spam',
       'ham', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'ham',
       'spam', 'spam', 'ham', 'spam', 'spam', 'ham', 'spam', 'spam',
       'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam',
       'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam',
       'spam', 'spam'], dtype='<U4')

In [40]:
print("Prediccion \n", y_pred)
print("\n Etiqueta Reales", y_test)

Prediccion 
 ['spam' 'spam' 'spam' 'spam' 'spam' 'spam' 'spam' 'spam' 'spam' 'spam'
 'ham' 'spam' 'spam' 'spam' 'spam' 'spam' 'ham' 'spam' 'spam' 'spam'
 'spam' 'spam' 'spam' 'ham' 'spam' 'spam' 'ham' 'spam' 'spam' 'ham' 'spam'
 'spam' 'spam' 'spam' 'spam' 'spam' 'spam' 'spam' 'spam' 'spam' 'spam'
 'spam' 'spam' 'spam' 'spam' 'spam' 'spam' 'spam' 'spam' 'spam']

 Etiqueta Reales ['spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'ham', 'spam', 'spam', 'spam', 'spam', 'spam', 'ham', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'ham', 'spam', 'spam', 'ham', 'spam', 'spam', 'ham', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam']


####  Evaluacion de resultados

In [41]:
from sklearn.metrics import accuracy_score
print("Accuracy: {:.3f}".format(accuracy_score(y_test, y_pred)))

Accuracy: 1.000


# 5.- Aumentando el DataSet

In [42]:
# Leer 20000 correos electronicos
X, y = create_prep_dataset("datasets/datasets/trec07p/full/index", 20000)


Parsing email: 20000

In [43]:
# Utilizamos 15 000 correos electronicos para entrenar el algoritmo y 5000 para realizar pruebas
X_train, y_train = X[:15000], y[:15000]
X_test, y_test = X[15000:], y[15000:]

In [44]:
vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(X_train)

In [46]:
clf = LogisticRegression()
clf.fit(X_train, y_train)

/home/jocelin/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [48]:
X_test = vectorizer.transform(X_test)
y_pred = clf.predict(X_test)

In [49]:
print("Accuracy: {:.3f}".format(accuracy_score(y_test, y_pred)))

Accuracy: 0.990
